In [ ]:
pip install holidays

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import time 
from datetime import datetime

import xgboost as xgb
from xgboost import plot_importance, plot_tree
import pandas_profiling

from sklearn.model_selection import train_test_split

import holidays

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import *

In [ ]:
holiday = holidays.HK()

In [ ]:
df = pd.read_csv('../input/msbd5001/train.csv')
dft = pd.read_csv('../input/msbd5001/test.csv')

In [ ]:
df['Date'] = df['date'].map(lambda x:datetime.strptime(x,"%d/%m/%Y %H:%M"))
dft['Date'] = dft['date'].map(lambda x:datetime.strptime(x,"%d/%m/%Y %H:%M"))

In [ ]:
df.drop(['id',"date"],axis=1,inplace=True)
dft.drop(['id',"date"],axis=1,inplace=True)

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df['Date'].dt.date
    df['hour'] = df['Date'].dt.hour
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofmonth'] = df['Date'].dt.day
    df['weekofyear'] = df['Date'].dt.weekofyear
    
    dates = list(df['date'])
    Is_Holiday=[]
    for i in dates:
        Is_Holiday.append(int(i in holiday))
    df['Is_Holiday']=Is_Holiday
    
    return df

In [ ]:
df=create_features(df)

In [ ]:
dft=create_features(dft)

In [ ]:
y = df['speed']

In [ ]:
X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','Is_Holiday']]

In [ ]:
xgb_model = xgb.XGBRegressor(nthread = 15)
cv_split = ShuffleSplit(n_splits=6, train_size=0.7, test_size=0.2)
param_grid = dict(
    max_depth = [4, 5, 6],
    learning_rate = np.linspace(0.03, 0.3, 10),
    n_estimators = [100, 200]
)
start = time.time()
grid = GridSearchCV(xgb_model, param_grid, cv=cv_split, scoring='neg_mean_squared_error')
grid.fit(X, y)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
dft['speed']=grid.predict(dft[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','Is_Holiday']])

In [ ]:
dft

In [ ]:
dft['id']=dft.index

In [ ]:
dft.drop(['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear','date','Date','Is_Holiday'], axis=1, inplace=True)

In [ ]:
dft

In [ ]:
dft=dft[['id', 'speed']]

In [ ]:
dft.to_csv('20123808.csv', index=False)